In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd


In [6]:
jobs_prompt = ['Analisi dei Dati','Vendite']
locations = ['Reggio','Verona']

job_head = []
job_detail = []

In [7]:
for job in jobs_prompt:
    for location in locations:
        job_name = job
        country_name = location
        filename = f"{job_name.lower().replace(' ', '-')}_{country_name.lower().replace(' ', '-')}_{time.strftime('%d-%m-%Y')}.csv"

        job_url = "";
        for item in job_name.split(" "):
            if item != job_name.split(" ")[-1]:
                job_url = job_url + item + "%20"
            else:
                job_url = job_url + item

        country_url = "";
        for item in country_name.split(" "):
            if item != country_name.split(" ")[-1]:
                country_url = country_url + item + "%20"
            else:
                country_url = country_url + item

        geoId = ""
        if "Amsterdam" in country_url:
            geoId = "103100785"
        elif "Berlin" in country_url:
            geoId = "106967730"
        elif "Barcelona" in country_url:
            geoId = "107025191"
        elif "Reggio" in country_url:
            geoId = "102687173"
        elif "Verona" in country_url:
            geoId = "102557975"   


        url = "https://www.linkedin.com/jobs/{0}-jobs-{1}?keywords={2}&location={3}&locationId=&geoId={4}&f_TPR=r86400&distance=25&position=1&pageNum=0".format(job_name.lower().replace(" ", "-"), country_name.lower().replace(" ", "-"), job_url, country_url, geoId)

        print(url)

https://www.linkedin.com/jobs/analisi-dei-dati-jobs-reggio?keywords=Analisi%20dei%20Dati&location=Reggio&locationId=&geoId=102687173&f_TPR=r86400&distance=25&position=1&pageNum=0
https://www.linkedin.com/jobs/analisi-dei-dati-jobs-verona?keywords=Analisi%20dei%20Dati&location=Verona&locationId=&geoId=102557975&f_TPR=r86400&distance=25&position=1&pageNum=0
https://www.linkedin.com/jobs/vendite-jobs-reggio?keywords=Vendite&location=Reggio&locationId=&geoId=102687173&f_TPR=r86400&distance=25&position=1&pageNum=0
https://www.linkedin.com/jobs/vendite-jobs-verona?keywords=Vendite&location=Verona&locationId=&geoId=102557975&f_TPR=r86400&distance=25&position=1&pageNum=0


In [8]:
#for urls in url:
# Creating a webdriver instance
browser = webdriver.Firefox()

# Opening the url we have just defined in our browser
browser.get(url)


In [9]:
#We find how many jobs are offered.1000
jobs_num = browser.find_element(By.CSS_SELECTOR,"h1>span").get_attribute("innerText")
if len(jobs_num.split(',')) > 1:
    jobs_num = int(jobs_num.split(',')[0])*1000
else:
    jobs_num = int(jobs_num)

jobs_num   = int(jobs_num)

#We create a while loop to browse all jobs. 
i = 2
while i <= int(jobs_num/2)+1:
    #We keep scrollind down to the end of the view.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    i = i + 1
    print("Current at: ", i, "Percentage at: ", ((i+1)/(int(jobs_num/2)+1))*100, "%",end="\r")
    try:
        #We try to click on the load more results buttons in case it is already displayed.
        infinite_scroller_button = browser.find_element(By.XPATH, ".//button[@aria-label='Load more results']")
        infinite_scroller_button.click()
        time.sleep(0.1)
    except:
        #If there is no button, there will be an error, so we keep scrolling down.
        time.sleep(0.1)
        pass

In [10]:
#We get a list containing all jobs that we have found.
job_lists = browser.find_element(By.CLASS_NAME,"jobs-search__results-list")
jobs_tags = job_lists.find_elements(By.TAG_NAME,"li") # return a list

#We declare void list to keep track of all obtaind data.
job_title_list = []
company_name_list = []
location_list = []
date_list = []
job_link_list = []

#We loof over every job_element and obtain all the wanted info.
for job_element in jobs_tags:
        #job_title
        job_title = job_element.find_element(By.CSS_SELECTOR,"h3").get_attribute("innerText")
        job_title_list.append(job_title)
        
        #company_name
        company_name = job_element.find_element(By.CSS_SELECTOR,"h4").get_attribute("innerText")
        company_name_list.append(company_name)
        
        #location
        location = job_element.find_element(By.CSS_SELECTOR,"div>div>span").get_attribute("innerText")
        location_list.append(location)
        
        #date
        date = job_element.find_element(By.CSS_SELECTOR,"div>div>time").get_attribute("datetime")
        date_list.append(date)
        
        #job_link
        job_link = job_element.find_element(By.CSS_SELECTOR,"a").get_attribute("href")
        job_link_list.append(job_link)

        print(job_title)
                
                #date_print = date
                #company_name_print = company_name
                #job_title_print = job_title
                #location_print = location
                #job_link_print = job_link

                #job_head = pd.DataFrame({
                #    'Date': date_print,
                #    'Company': company_name_print,
                #    'Title': job_title_print,
                #    'Location': location_print,
                #    'Link': job_link
                #})

                ##print(job_head)
                #job_head.to_csv('jupiter.csv', index=False)

AREA SALES MANAGER – Polonia e Est Europa
Fronius Italia: Candidatura Spontanea generica
Sales Officer
AREA MANAGER NORD EST PER IL SETTORE EDILE
Addetto/a Vendite | Legnago
Regional Sales Manager Central Europe
Impiegato Supporto Tecnico - Tech Support National
Addetto Vendite part-time appartenente alle categorie protette (Legge 68/99) (f/m)
Addetto Vendite part-time appartenente alle categorie protette (Legge 68/99) (f/m)
Brand Manager - Luxury
Technical Support National Solar Energy - impiegato Supporto Tecnico
Customer care
Responsabile di Negozio _ Verona
Event Marketing & Comunicazione - part time e full time - anche prima esperienza
Ausiliare Vendite (m/f/x) Categorie Protette (L. 68/99) - Filiale di Verona, Via Mameli (VR)
Capo Contabile (M/F)
Consulente sap senior


In [11]:
browser.quit()

TypeError: 'WebDriver' object is not iterable